In [ ]:
import ee
import geepy

import branca.colormap as cm
import random

ee.Initialize()

In [ ]:
config = geepy.params.configParams('input.json')

ls = ee.FeatureCollection(config.params['fusionTable'],'geometry')
tiles = [geepy.feature.getTiles(ls, tile, 'TILE_T') for tile in config.params['grid']['tiles2process']]

l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['id']).select(
    config.params["imgCollection"]["lc5"]["bands"],
    config.params["imgCollection"]["lc5"]["bandNames"])

l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['id']).select(
    config.params["imgCollection"]["lc7"]["bands"],
    config.params["imgCollection"]["lc7"]["bandNames"])

l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['id']).select(
    config.params["imgCollection"]["lc8"]["bands"],
    config.params["imgCollection"]["lc8"]["bandNames"])

In [ ]:
tileNames = [tiles[i][0] for i in range(len(tiles))]
tileCoords = [tiles[i][1].geometry().getInfo()['coordinates'] for i in range(len(tiles))]

In [ ]:
for year in range(2015, 2016):
    y = str(year)
    start_d = config.params['years'][y]+config.params['period']['dry']['start']
    end_d = config.params['years'][y]+config.params['period']['dry']['end']
    #print("Start-End dry season %s %s" %(start_d, end_d))
    
    if(year < 2002):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', 20).filterDate(start_d, end_d)
    elif(year in (2002,2011,2012)):
        filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', 20).filterDate(start_d, end_d)
    elif(year > 2002 and year < 2011):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', 20).filterDate(start_d, end_d)
    else:
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', 20).filterDate(start_d, end_d)

    filt = filtered.median()
    
    #Landsat Tiles
    lsClip = [geepy.image.clipImageCollection(filt, tiles[i][1]) for i in range(len(tiles))]

In [ ]:
for i in range(len(lsClip)):
    lsClip[i] = geepy.image.calcNDVI(lsClip[i])
    lsClip[i] = geepy.image.calcEVI(lsClip[i])
    lsClip[i] = geepy.image.calcSAVI(lsClip[i])
    lsClip[i] = geepy.image.calcNDWI(lsClip[i])
    lsClip[i] = geepy.image.waterMask(lsClip[i])
    lsClip[i] = geepy.image.calcFractions(lsClip[i])
    lsClip[i] = geepy.image.calcNDFI(lsClip[i])
    lsClip[i] = geepy.image.calcNDFI2(lsClip[i])
    lsClip[i] = geepy.image.calcNDFI3(lsClip[i])
    lsClip[i] = geepy.image.calcFCI(lsClip[i])
    lsClip[i] = geepy.image.calcGVNPV(lsClip[i])
    lsClip[i] = geepy.image.calcNPVSOIL(lsClip[i])

In [ ]:
# dados do mapbiomas
# Coleção 2.3
mapBiomas = ee.Image("projects/mapbiomas-workspace/public/collection2_3/mapbiomas_collection23_integration_v1")

In [ ]:
for i in range(2000, 2016):
    dt = 'classification_'+ str(i)
    
    mbClip = [geepy.image.clipImageCollection(mapBiomas.select(dt), tiles[i][1]) for i in range(len(tiles))]

In [ ]:
for i in range(len(mbClip)):
    mbClip[i] = mbClip[i].addBands(ee.Image.pixelLonLat())

In [ ]:
for i in range(len(mbClip)):
    mbClip[i] = mbClip[i].rename(["class", "longitude", "latitude"]).cast({"class": "int8"})

In [ ]:
#lsClip[tileIndex].bandNames().getInfo()
tileIndex = 11
lsClip[tileIndex].bandNames().getInfo()

In [ ]:
mapa_c = geepy.maps.geeMap(tiles[tileIndex][1],zoom=10)

# mapa_c.addLayer(lsClip[tileIndex].select('blue'), name='blue')
# mapa_c.addLayer(lsClip[tileIndex].select('green'), name='green')
# mapa_c.addLayer(lsClip[tileIndex].select('red'), name='red')
# mapa_c.addLayer(lsClip[tileIndex].select('nir'), name='nir')
# mapa_c.addLayer(lsClip[tileIndex].select('swir1'), name='swir1')
# mapa_c.addLayer(lsClip[tileIndex].select('swir2'), name='swir2')
# mapa_c.addLayer(lsClip[tileIndex].select('thermal'), name='thermal')
# mapa_c.addLayer(lsClip[tileIndex].select('fmask'), name='fmask')
# mapa_c.addLayer(lsClip[tileIndex].select('ndvi'), name='ndvi')
# mapa_c.addLayer(lsClip[tileIndex].select('evi'), name='evi')
# mapa_c.addLayer(lsClip[tileIndex].select('savi'), name='savi')
# mapa_c.addLayer(lsClip[tileIndex].select('ndwi'), name='ndwi')
# mapa_c.addLayer(lsClip[tileIndex].select('wmask'), name='wmask')
# mapa_c.addLayer(lsClip[tileIndex].select('gv'), name='gv')
# mapa_c.addLayer(lsClip[tileIndex].select('npv'), name='npv')
# mapa_c.addLayer(lsClip[tileIndex].select('soil'), name='soil')
# mapa_c.addLayer(lsClip[tileIndex].select('cloud'), name='cloud')
# mapa_c.addLayer(lsClip[tileIndex].select('gvs'), name='gvs')
# mapa_c.addLayer(lsClip[tileIndex].select('shade'), name='shade')
# mapa_c.addLayer(lsClip[tileIndex].select('ndfi'), name='ndfi')
# mapa_c.addLayer(lsClip[tileIndex].select('fci'), name='fci')

mapa_c.addLayer(lsClip[tileIndex].select('ndfi2'), name='ndfi2')
mapa_c.addLayer(lsClip[tileIndex].select('ndfi3'), name='ndfi3')
mapa_c.addLayer(lsClip[tileIndex].select('gvnpv'), name='gvnpv')
mapa_c.addLayer(lsClip[tileIndex].select('npvsoil'), name='npvsoil')

mapa_c.addControls()

In [ ]:
                            #  '006400,00ff00,e974ed,b2f734,ff0000,0000ff'
colormap = cm.LinearColormap(['#006400','#00ff00','#e974ed','#b2f734','#ff0000','#0000ff'],
    vmin=1,
    vmax=6
).to_step(6)


colormapbiomas = cm.LinearColormap(['#d5d5e5','#129912','#1f4423','#006400','#00ff00','#687537',\
                    '#76a5af','#29eee4','#77a605','#935132','#ff9966','#45c2a5', \
                    '#b8af4f','#f1c232','#ffffb2','#ffd966','#f6b26b','#a0d0de', \
                    '#e974ed','#d5a6bd','#c27ba0','#FBF3C7','#ea9999','#dd7e6b', \
                    '#b7b7b7','#ff99ff','#0000ff','#d5d5e5','#ce3d3d',],
    vmin=0,
    vmax=28
).to_step(29)

colormapbiomas

In [ ]:
colormap

In [ ]:
task = geepy.image.send2drive(remaped, tileCoords[tileIndex], 'class_erroUrbano', 'driveDir')

In [ ]:
task.start()

In [ ]:
task = geepy.image.send2drive(classified_rf, tileCoords[tileIndex], 'class_new_27', 'driveDir')

In [ ]:
task.start()